In [11]:
%matplotlib widget

In [19]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sin, cos, acos, tan, atan, radians 

In [109]:
def define_circle(p1, p2, p3):
    """
    Returns the center and radius of the circle passing the given 3 points.
    In case the 3 points form a line, returns (None, infinity).
    """
    temp = p2[0] * p2[0] + p2[1] * p2[1]
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])

    if abs(det) < 1.0e-6:
        return (None, np.inf)

    # Center of circle
    cx = (bc*(p2[1] - p3[1]) - cd*(p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det

    radius = np.sqrt((cx - p1[0])**2 + (cy - p1[1])**2)
    return ((cx, cy), radius)

In [154]:
m = 1.0            # module
z = 60             # number of teeth
a0 = radians(20.0) # pressure angle
c = 0.0           # clearance

d0 = m * z         # pitch diameter
da = m * (z + 2.0) # addendum circle diameter
dd = m * (z - 2.0) - 2.0 * c # dedendum circle diameter
s0 = m * (pi / 2.0 + 0.0 * tan(a0)) # tooth thickness on the pitch circle
inv_a0 = tan(a0) - a0

r0 = d0 / 2.0
ra = da / 2.0
rd = dd / 2.0

npoints = 300

t = np.linspace(0.0, pi * 2.0, npoints)

db = np.cos(a0) * d0
d = np.linspace(db, da, npoints)

cos_a = d0 / d * np.cos(a0)
# cos_a[np.where(cos_a > 1.0)] = 1.0
a = np.arccos(cos_a)
inv_a = np.tan(a) - a
s = d * (s0 / d0 + inv_a0 - inv_a)
phi = (s / 2.0) / (d / 2.0)

def gear_points():
    curve_points = 20
    
    rb = np.cos(a0) * d0 / 2.0 # base circle radius
    r = np.linspace(rb, ra, curve_points)
    cos_a = r0 / r * np.cos(a0)
    a = np.arccos(cos_a)
    inv_a = np.tan(a) - a
    s = r * (s0 / d0 + inv_a0 - inv_a)
    phi = s / r
    
    x = np.cos(phi) * r
    y = np.sin(phi) * r
    
    b = np.linspace(phi[-1], -phi[-1], curve_points)
    x = np.concatenate((x, np.cos(b) * ra))
    y = np.concatenate((y, np.sin(b) * ra))
    
    x = np.concatenate((x, (np.cos(-phi) * r)[::-1]))
    y = np.concatenate((y, (np.sin(-phi) * r)[::-1]))
    
    tau = np.pi * 2.0 / z
    rho = tau - phi[0] * 2.0
    
    p1 = (x[-1], y[-1])
    p2 = (np.cos(-phi[0] - rho / 2.0) * rd,
          np.sin(-phi[0] - rho / 2.0) * rd)
    p3 = (np.cos(-phi[0] - rho) * rb,
          np.sin(-phi[0] - rho) * rb)
    
    bcxy, bcr = define_circle(p1, p2, p3)
        
    t1 = np.arctan2(p1[1] - bcxy[1], p1[0] - bcxy[0])
    t2 = np.arctan2(p3[1] - bcxy[1], p3[0] - bcxy[0])
    
    if t1 < 0.0:
        t1 += np.pi * 2.0
    if t2 < 0.0:
        t2 += np.pi * 2.0
        
    t1, t2 = min(t1, t2), max(t1, t2)
    
    t = np.linspace(t1 + np.pi * 2.0, t2 + np.pi * 2.0, curve_points)
    
    x = np.concatenate((x, bcxy[0] + bcr * np.cos(t)))
    y = np.concatenate((y, bcxy[1] + bcr * np.sin(t)))
    
    res_x, res_y = x.copy(), y.copy()
    
    angle = -tau
    for i in range(z - 1):
        res_x = np.concatenate((res_x, x * np.cos(angle) - y * np.sin(angle)))
        res_y = np.concatenate((res_y, x * np.sin(angle) + y * np.cos(angle)))
        angle -= tau
        
    return res_x, res_y

fig = plt.figure()
plt.plot(np.cos(t) * r0, np.sin(t) * r0)
plt.plot(np.cos(t) * ra, np.sin(t) * ra)
plt.plot(np.cos(t) * rd, np.sin(t) * rd)

gx, gy = gear_points()

plt.plot(gx, gy)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [133]:
x = np.arange(10)
x.copy()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])